補足資料 画像の集め方

１、icrawlerを使おう

画像データセットを作るのに「よし！今からブラウザ開いてgoogle画像漁るぞ!」なんてやってたらキリがありません。プログラムで自動化したいですよね？そこで、今回はicrawlerというライブラリを用いて、効率よく画像を集めていきます。

このicrawlerとは「Bing」や「Chrome」といったブラウザの画像を簡単に集めてくれるライブラリです。今回は「Chrome」に載ってる画像をゲットするために「GoogleImageCrawler」というものを使っていきます。ついでに、こういったインターネットにあるコンテンツやデータを集めてくるプログラムの事を、「クローラー」といいます。

In [2]:
%pip install icrawler 
%pip install opencv-python
from icrawler.builtin import GoogleImageCrawler as GI #長いので略す

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


これで、icrawlerのインストールとインポートは完了です。ん？opencv-pythonというのもインストールされてますね。これは後で説明します

それでは実際に画像を集めてみます

In [1]:
%mkdir images
#まず、画像群をしまっておくフォルダを制作

�T�u�f�B���N�g���܂��̓t�@�C�� images �͊��ɑ��݂��܂��B


In [3]:
#GoogleImageCrawlerのインスタンスと定義。画像を「imagesの中のdogフォルダに保存する」
crawler=GI(storage={"root_dir":"images/dog"})
#キーワードを「犬」として検索。最大50枚の画像をゲットする。
crawler.crawl(keyword="いぬ",max_num=50)

2023-06-06 18:20:09,965 - INFO - icrawler.crawler - start crawling...
2023-06-06 18:20:09,966 - INFO - icrawler.crawler - starting 1 feeder threads...
2023-06-06 18:20:09,968 - INFO - feeder - thread feeder-001 exit
2023-06-06 18:20:09,971 - INFO - icrawler.crawler - starting 1 parser threads...
2023-06-06 18:20:09,975 - INFO - icrawler.crawler - starting 1 downloader threads...
2023-06-06 18:20:10,790 - INFO - parser - parsing result page https://www.google.com/search?q=%E3%81%84%E3%81%AC&ijn=0&start=0&tbs=&tbm=isch
2023-06-06 18:20:11,043 - INFO - downloader - image #1	https://img.benesse-cms.jp/item/image/user/7c9fe6831f52e30e0ede4f8c54fd9bba673e8d8b/39bc4015-12b4-48c7-9c47-5d1998b63122.jpg
2023-06-06 18:20:11,401 - INFO - downloader - image #2	https://skywardplus.jal.co.jp/wp-content/uploads/2020/07/eyecatch_dogday_main.jpg
2023-06-06 18:20:11,575 - INFO - downloader - image #3	https://book.impress.co.jp/upload/page-image/1119102022_IN01.jpg
2023-06-06 18:20:12,153 - ERROR - downlo

2023-06-06 18:20:24,828 - INFO - parser - downloaded image reached max num, thread parser-001 is ready to exit
2023-06-06 18:20:24,828 - INFO - parser - thread parser-001 exit


すこし時間はかかりますが手よりはマシです。ではimages/dogに画像が保存されているか確認してみてください。

つぎは、猫の画像を集めてみましょう。

In [4]:
crawler=GI(storage={"root_dir":"images/cat"})
crawler.crawl(keyword="猫",max_num=50)

2023-06-06 18:20:30,417 - INFO - icrawler.crawler - start crawling...
2023-06-06 18:20:30,418 - INFO - icrawler.crawler - starting 1 feeder threads...
2023-06-06 18:20:30,420 - INFO - feeder - thread feeder-001 exit
2023-06-06 18:20:30,423 - INFO - icrawler.crawler - starting 1 parser threads...
2023-06-06 18:20:30,426 - INFO - icrawler.crawler - starting 1 downloader threads...
2023-06-06 18:20:31,276 - INFO - parser - parsing result page https://www.google.com/search?q=%E7%8C%AB&ijn=0&start=0&tbs=&tbm=isch
2023-06-06 18:20:31,467 - INFO - downloader - image #1	https://dol.ismcdn.jp/mwimgs/5/d/600/img_7e0b8adba77c91687a8078920dedc7be160077.jpg
2023-06-06 18:20:32,098 - ERROR - downloader - Response status code 404, file https://upload.wikimedia.org/wikipedia/commons/thumb/a/a4/A_cat_on_a_motorcycle_in_the_medina_of_Tunis_20171017_131525.jpg
2023-06-06 18:20:32,230 - INFO - downloader - image #2	https://fujifilmsquare.jp/assets/img/column/column_24_mv.jpg
2023-06-06 18:20:32,468 - INFO

2023-06-06 18:20:43,307 - INFO - parser - downloaded image reached max num, thread parser-001 is ready to exit
2023-06-06 18:20:43,307 - INFO - parser - thread parser-001 exit


これで、画像自体の準備は完了です。

しかし、このクローラー、ちょっとした問題があります。それは、画像ファイルの拡張子がバラバラになっている可能性がある点です。これは、学習時にエラーを吐く大きな原因となってしまうので、何とかして一つの拡張子に統一しなきゃですね。ここでは.jpgへの統一を目標としましょう。

2、画像を一気に.jpgに

それでは、さっきゲットしてきた画像たちを一気に.jpgにしちゃいましょう。microsoftのサービスとして、特定の画像を.jpgに変換してくれるものがありますが、これはひたすらドラッグアンドドロップなので時間がかかりすぎです。よって、opencvとglob、osを用いて、一気にやっちゃいます。

まずは、globの出番です。このglobというライブラリは、フォルダやファイルのパスの取得に物凄く有用なライブラリです。以下のコードは、さっきダウンロードした画像たちのパスをすべて読み込んで、変数に代入しています。

In [5]:
import glob
files_dog=glob.glob("images/dog/*") #"*"と示すことで、フォルダの中のすべてのファイルのパスをゲットできる
files_cat=glob.glob("images/cat/*")

print(files_dog)

['images/dog\\000001.jpg', 'images/dog\\000002.jpg', 'images/dog\\000003.jpg', 'images/dog\\000004.jpg', 'images/dog\\000005.jpg', 'images/dog\\000006.jpg', 'images/dog\\000007.jpg', 'images/dog\\000008.jpg', 'images/dog\\000009.jpg', 'images/dog\\000010.jpg', 'images/dog\\000011.jpg', 'images/dog\\000012.jpg', 'images/dog\\000013.jpg', 'images/dog\\000014.jpg', 'images/dog\\000015.jpg', 'images/dog\\000016.jpg', 'images/dog\\000017.jpg', 'images/dog\\000018.jpg', 'images/dog\\000019.jpg', 'images/dog\\000020.jpg', 'images/dog\\000021.jpg', 'images/dog\\000022.jpg', 'images/dog\\000023.jpg', 'images/dog\\000024.jpg', 'images/dog\\000025.jpg', 'images/dog\\000026.jpg', 'images/dog\\000027.jpg', 'images/dog\\000028.jpg', 'images/dog\\000029.jpg', 'images/dog\\000030.jpg', 'images/dog\\000031.jpg', 'images/dog\\000032.jpg', 'images/dog\\000033.jpg', 'images/dog\\000034.jpg', 'images/dog\\000035.jpg', 'images/dog\\000036.jpg', 'images/dog\\000037.jpg', 'images/dog\\000038.jpg', 'images/dog

このように、保存された画像の相対パスがすべて配列に格納されています。

それでは、このパス達を用いて、すべての画像を.jpgに変換していきます。ここで、opencvの出番です。

opencvとは、画像処理に用いられるオープンソースのライブラリです。以下のコードは、画像データを.jpgに変換し、変換前の画像を削除するという処理を行っています。

In [6]:
import cv2 #opencvはcv2として読み込まれる
import os

#さっきのパス達の配列を保存する配列を生成
files=[files_dog,files_cat]

for i in files:
    for j in i:

        #cv2.imreadで、指定されたパスの画像を読み込む
        img=cv2.imread(j)

        #osライブラリは、osの機能をpythonで使うためのもの
        #さっき読み込んだ画像を削除している（データはimg変数に残ってるので安心）
        os.remove(j)

        #cv2.imwriteで画像に名前を付けて保存する。
        #読み込んだファイルの拡張子を.jpgにして、データもjpeg形式にして保存している
        cv2.imwrite(j.split(".")[0]+".jpg",img,[cv2.IMWRITE_JPEG_QUALITY,200]) #この200は、画像の鮮明さを示すらしい

このコードだと、もともと.jpgだったファイルも読み込んでしまいますが、特に問題なしです。

以上で、画像の準備は完了です。お疲れ様でした。つぎは、「csv_base」に取り組んでください